In [1]:
import numpy as np

# 模块0：转换标准形式  但是好像也用不到,那就先不写
def convert_to_standard_form(c, A, b):
    """
    
    """
    index = []
    for i in range(len(b)):
        if np.all(A[i,:]==0):
            index.append(i)
    A = np.delete(A, index, axis=0)
    b = np.delete(b, index, axis=0)
    return c, A, b

In [2]:
# 定义一个线性规划问题
c = np.array([-5, -3, 1])  # 目标函数系数
A = np.array([[1, 2, 3], [2, 4, 6]])  # 约束矩阵
b = np.array([2, 4])  # 右侧常数

# 模块0：转换标准形式
c_std, A_std, b_std = convert_to_standard_form(c, A, b)
print(c_std,'\n',A_std,'\n','\n',b_std)

[-5 -3  1] 
 [[1 2 3]
 [2 4 6]] 
 
 [2 4]


In [3]:
# 模块1：检查A是否行满秩，移除多余的约束， 这里检查(A,b)是否满秩
def remove_redundant_constraints(A, b): 
    """
    检查并移除冗余约束，返回去冗余后的 A 和 b
    使用方法2，但方法2真的行吗
    """
    A = np.column_stack((A,b))
    # print(A)
    q,r = np.linalg.qr(A.T)
    index = np.abs(np.diag(r ) )> 1e-6 # 得到正常的数组
    B = (A.T[:,index]).T
    return B[:,:-1], B[:,-1]  # 直接也解决了系数全为0的问题

print(remove_redundant_constraints(A,b))
A ,b = remove_redundant_constraints(A,b)

(array([[1, 2, 3]]), array([2]))


In [4]:
# 模块2：初始化可行基解：大M法
def initialize_feasible_solution(A, b, c):
    """
    使用大M法初始化一个可行基解
    """
    m, n = A.shape
    M = 1e6  # 极大的常数M
    artificial_vars = np.eye(m) # 人工变量的系数
    
    # 增加人工变量
    A_aug = np.hstack([A, artificial_vars])
    c_aug = np.hstack([c, M * np.ones(m)])

    # 初始化人工变量
    x1 = np.copy(b) # 人工变量的地址
    x0 = np.zeros(n)
    init_basic = np.concatenate([x0,x1])
    return A_aug, c_aug, init_basic

In [5]:
A_aug, c_aug, init_basic=initialize_feasible_solution(A,b, c)
print(A_aug, c_aug, init_basic)
print(b)

[[1. 2. 3. 1.]] [-5.e+00 -3.e+00  1.e+00  1.e+06] [0. 0. 0. 2.]
[2]


In [6]:

# 模块3：单纯形法的迭代
def simplex_method(A, b, c, initial_basic):
    """
    单纯形法的迭代求解
    """
    if len(A.shape) == 1:
        A = A.reshape(1, -1)
    m, n = A.shape
    # 初始化基解
    x = initial_basic
    # non_basic_vars = np.arange(n)
    var_index = np.arange((n-m),n) 
    table = np.column_stack((A,b)) # 构建一张表,便于计算
    zs = np.zeros(n+1)
    for i in range(n):
        zs[i] = c[i] - np.dot(c[var_index], table[:,i])
    zs[n] = -np.dot(c[var_index], table[:,n])# 不去进行合并

    while True:
        in_index = np.argmin(zs[:-1]) # 找到系数最小的 ,但是不能是最后一个
        if zs[in_index] >= 0:
            x = np.zeros(n)
            for i in range(m):
                x[var_index[i]] = table[i,-1]
            return x
        t = np.array([table[i,-1] / table[i,in_index] if(abs(table[i,in_index])>1e-6) else 1e6 for i in range(m) ])
        out_index = np.argmin(t) # var_index 中的, 以此为基础
        var_index[out_index] = in_index

        p = table[out_index, in_index] 
        table[out_index,:] = table[out_index,:] / p
        for i in range(m):
            if i == out_index:
                continue
            table[i] = table[i] - table[i,in_index] * table[out_index,:] # 处理每一行
        # 更新zs 也即是temp 
        for i in range(n):
            zs[i] = c[i] - np.dot(c[var_index], table[:,i])
        zs[n] = -np.dot(c[var_index], table[:,n]) # 不去进行合并    

simplex_method(A_aug,b, c_aug, init_basic)


array([2., 0., 0., 0.])

In [7]:
import numpy as np

def simplex_method_with_bland(A, b, c, initial_basic):
    """
    单纯形法的迭代求解（带Bland's rule）
    """
    m, n = A.shape
    # 初始化基解
    x = np.zeros(n)  # 初始化解向量
    var_index = np.arange(n - m, n)  # 基变量的索引
    table = np.column_stack((A, b))  # 构建单纯形表格
    zs = np.zeros(n + 1)

    # 初始化目标函数系数
    for i in range(n):
        zs[i] = c[i] - np.dot(c[var_index], table[:, i])
    zs[n] = -np.dot(c[var_index], table[:, n])  # 更新目标函数常数项

    while True:
        # 1. 寻找最小的系数（入基变量）
        in_index_candidates = np.where(zs[:-1] < 0)[0]  # 选择 c_i < 0 的所有非基变量
        if len(in_index_candidates) == 0:  # 没有负值，说明已经找到了最优解
            for i in range(m):
                x[var_index[i]] = table[i, -1]
            return x
        
        # 选择最小的下标作为入基变量（Bland's rule）
        in_index = min(in_index_candidates)

        # 2. 计算比率（离基变量）
        t = np.array([table[i, -1] / table[i, in_index] if abs(table[i, in_index]) > 1e-6 else 1e6 for i in range(m)])
        out_index_candidates = np.where(t == np.min(t))[0]  # 找到所有比率相同的行
        out_index = min(out_index_candidates)  # 选择最小下标作为出基变量（Bland's rule）

        # 3. 更新基变量和非基变量
        var_index[out_index] = in_index

        # 4. 对表格进行行变换，更新入基变量所在列
        p = table[out_index, in_index]  # pivot element
        table[out_index, :] /= p  # 将出基变量所在行除以pivot

        # 对其他行进行高斯消元
        for i in range(m):
            if i != out_index:
                table[i, :] -= table[i, in_index] * table[out_index, :]

        # 5. 更新zs向量
        for i in range(n):
            zs[i] = c[i] - np.dot(c[var_index], table[:, i])
        zs[n] = -np.dot(c[var_index], table[:, n])  # 更新目标函数常数项

    return x

A_aug = np.array([[1., 2., 3. ,1., 0. ],
                  [2., 4. ,5., 0., 1.,]]) 
c_aug = np.array([-5.e+00 ,-3.e+00, 2, 1.e+06 ,1.e+06]) 
init_basic =np.array([0., 0., 0., 2., 3.])
b = np.array([2, 3 ])
x = simplex_method_with_bland(A_aug,b, c_aug, init_basic)
print(x)

[0.  0.  0.6 0.2 0. ]
